In [80]:
from functools import wraps

from htools import assert_raises, auto_repr

In [31]:
def delegate(attr):
    """Decorator that automatically delegates attribute calls to an attribute
    of the class. This is a nice convenience to have when using composition.
    This does NOT affect magic methods; for that, see the `forwardable`
    library.
    
    Note: I suspect this could lead to some unexpected behavior so be careful
    using this in production.
    
    Parameters
    ----------
    attr: str
        Name of variable to delegate to.
    
    Examples
    --------
    Example 1: We can use BeautifulSoup methods like `find_all` directly on 
    the Page object. Most IDEs should let us view quick documentation as well.
    
    @delegate('soup')
    class Page:
        def __init__(self, url, logfile, timeout):
            self.soup = self.fetch(url, timeout=timeout)
        ...
        
    page = Page('http://www.coursera.org')
    page.find_all('div')
    
    Example 2: Magic methods are not delegated.
    
    @delegate('data')
    class Foo:
        def __init__(self, data, city):
            self.data = data
            self.city = city
            
    >>> f = Foo(['a', 'b', 'c'], 'San Francisco')
    >>> len(f)
    
    TypeError: object of type 'Foo' has no len()
    """
    def wrapper(cls):
        def f(self, new_attr):
            delegate = getattr(self, attr)
            return getattr(delegate, new_attr)
        cls.__getattr__ = f
        return cls
    return wrapper

In [32]:
@delegate('arr')
class Foo:
    def __init__(self, a, b, arr, verbose=True):
        self.a, self.b = a, b
        self.arr = arr
        self.verbose = verbose
    def walk(self):
        return 'walking'
    def __getitem__(self, i):
        return list(range(self.a))[i]

In [33]:
f = Foo(9, 2, list('defghijkl'))
f

In [34]:
f.a, f.b, f.arr, f.verbose

(9, 2, ['d', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l'], True)

In [35]:
f.append(99)

In [36]:
f.arr

['d', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 99]

In [37]:
f.pop(-1)

99

In [38]:
f.sort()

In [39]:
f

In [40]:
f.arr

['d', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']

In [41]:
f[7]

7

In [42]:
'c' in f, 'd' in f, 'j' in f, 3 in f, 2 in f, 9 in f

(False, False, False, True, True, False)

In [43]:
@delegate('data')
class Foo:
    def __init__(self, data, city):
        self.data = data
        self.city = city

f = Foo(['a', 'b', 'c'], 'San Francisco')
with assert_raises(TypeError):
    len(f)

As expected, got TypeError(object of type 'Foo' has no len()).


In [231]:
def new_delegate(attr, iter_magics=False, skip=(), getattr_=True):
    def wrapper(cls):
        def _delegate(self, attr):
            """Helper that retrieves object that an instance delegates to."""
            return getattr(self, attr)
        
        # Changes __getattr__: any attribute that is not an instance variable
        # will be delegated.
        if getattr_:
            def _getattr(self, new_attr):
                return getattr(_delegate(self, attr), new_attr)
            cls.__getattr__ = _getattr

        # If specified, delegate magic methods to make cls iterable.
        if iter_magics:
            if '__getitem__' not in skip: 
                def _getitem(self, i):
                    return _delegate(self, attr)[i]
                setattr(cls, '__getitem__', _getitem)
            
            if '__setitem__' not in skip: 
                def _setitem(self, i, val):
                    _delegate(self, attr)[i] = val
                setattr(cls, '__setitem__', _setitem)
            
            if '__delitem__' not in skip:
                def _delitem(self, i):
                    del _delegate(self, attr)[i]
                setattr(cls, '__delitem__', _delitem)
            
            if '__len__' not in skip:
                def _len(self):
                    return len(_delegate(self, attr))
                setattr(cls, '__len__', _len)
        return cls
    return wrapper

In [232]:
@new_delegate('data', True)
@auto_repr
class Foo:
    def __init__(self, data, city):
        self.data = data
        self.city = city

In [233]:
arr = ['a', 'b', 'c']
f = Foo(arr, 'San Francisco')
len(f)

3

In [234]:
f[1]

'b'

In [235]:
f[1] = 'd'

In [236]:
f

Foo(data=['a', 'd', 'c'], city='San Francisco')

In [237]:
for char in f:
    print(char)

a
d
c


In [238]:
arr

['a', 'd', 'c']

In [239]:
f.clear()

In [240]:
f

Foo(data=[], city='San Francisco')

In [241]:
arr

[]

In [242]:
arr2 = ['x', 'y', 'z', 'n']
f2 = Foo(arr2, 'LA')
f2

Foo(data=['x', 'y', 'z', 'n'], city='LA')

In [243]:
f

Foo(data=[], city='San Francisco')

In [244]:
f.append(3)
f, f2

(Foo(data=[3], city='San Francisco'),
 Foo(data=['x', 'y', 'z', 'n'], city='LA'))

In [245]:
f2.insert(1, 'r')
f, f2

(Foo(data=[3], city='San Francisco'),
 Foo(data=['x', 'r', 'y', 'z', 'n'], city='LA'))

In [246]:
del f2[-1]

In [247]:
f, f2

(Foo(data=[3], city='San Francisco'),
 Foo(data=['x', 'r', 'y', 'z'], city='LA'))

In [248]:
arr, arr2

([3], ['x', 'r', 'y', 'z'])

In [249]:
arr.insert(0, 99)
arr, arr2

([99, 3], ['x', 'r', 'y', 'z'])

In [250]:
f, f2

(Foo(data=[99, 3], city='San Francisco'),
 Foo(data=['x', 'r', 'y', 'z'], city='LA'))

In [274]:
@new_delegate('data')
@auto_repr
class Foo:
    def __init__(self, data, city):
        self.data = data
        self.city = city

In [275]:
arr = ['a', 'b', 'c']
f = Foo(arr, 'San Francisco')
with assert_raises(TypeError):
    len(f)

As expected, got TypeError(object of type 'Foo' has no len()).


In [276]:
with assert_raises(TypeError):
    f[1]

As expected, got TypeError('Foo' object is not subscriptable).


In [277]:
with assert_raises(TypeError):
    f[0] = 333

As expected, got TypeError('Foo' object does not support item assignment).


In [279]:
f

Foo(data=['a', 'b', 'c'], city='San Francisco')

In [280]:
arr.append('zzz')
arr, f

(['a', 'b', 'c', 'zzz'],
 Foo(data=['a', 'b', 'c', 'zzz'], city='San Francisco'))

In [281]:
f.data.insert(0, 'd')

In [282]:
f

Foo(data=['d', 'a', 'b', 'c', 'zzz'], city='San Francisco')

In [283]:
arr

['d', 'a', 'b', 'c', 'zzz']

In [284]:
f.clear()

In [285]:
f

Foo(data=[], city='San Francisco')

In [286]:
arr

[]

In [261]:
@new_delegate('data', True, ('__delitem__'))
@auto_repr
class Foo:
    def __init__(self, data, city):
        self.data = data
        self.city = city

In [262]:
arr = [55, 33, 11]
f = Foo(arr, 'NY')
f

Foo(data=[55, 33, 11], city='NY')

In [263]:
for n in f:
    print(n)

55
33
11


In [264]:
len(f)

3

In [265]:
f.pop(-1)

11

In [266]:
f, arr

(Foo(data=[55, 33], city='NY'), [55, 33])

In [267]:
f[0] = 99

In [268]:
f, arr

(Foo(data=[99, 33], city='NY'), [99, 33])

In [270]:
with assert_raises(AttributeError):
    del f[0]

As expected, got AttributeError(__delitem__).
